In [1]:
%matplotlib widget

In [2]:
import numpy as np
import xarray as xr
import copy

from scipy.spatial import Delaunay
from scipy.spatial.transform import Rotation as rot

# plotting
import matplotlib.pyplot as plt
import pims
import ipyvolume as ipv

# interactive plots
import ipywidgets as widgets
from ipywidgets import VBox, HBox, IntSlider, interactive_output, Label, Layout
from IPython.display import display


import libo
from libo.io import tc3

import basic_meshes as bm
from experimental_setup import get_setup

import weldx.geometry as geo
import weldx.transformations as tf
import weldx.utility as ut
import weldx.visualization as vs

# Setup

In [3]:
setup = get_setup(5)
target_frame_rate = 4

# Import and process experimental data

### Temperature data

In [4]:
dsx_sensor_data = tc3.to_xarray(tc3.read_db(setup.measurement_data_id))
dsx_temperature_data = dsx_sensor_data[["MH24_T01", "MH24_T02", "MH24_T03"]].dropna("time")

### Machine data

In [5]:
# import machine data
dsx_machine_data = tc3.to_xarray(tc3.read_db(setup.system_data_index))


# collect relevant data
relevant_data_names = ["FB_X", "FB_Y", "FB_Z", "FB_Rx", "FB_Ry", "FB_Rz", "trigSchweissen", "trigScan2_prog"]
dsx_relevant_machine_data = dsx_machine_data[relevant_data_names].dropna("time")
interpolated_temperature_data = dsx_temperature_data.interp_like(dsx_relevant_machine_data)
dsx_relevant_machine_data = xr.merge([interpolated_temperature_data, dsx_relevant_machine_data])


# extract system movement data
dsx_machine_position_data = dsx_relevant_machine_data[["FB_X", "FB_Y", "FB_Z", "FB_Rx", "FB_Ry", "FB_Rz"]]
machine_coordinates_base = dsx_machine_position_data.to_array().data[:3, :]
machine_angles_degree_base = dsx_machine_position_data.to_array().data[3:6, :]

### Scanner data

In [6]:
num_scans = len(setup.scan_data_scan_idx)
list_dsx_scan_data = []
for i in range(num_scans):
    data = libo.io.tools.merge_scan_tcp(scan=setup.scan_data_scan_idx[i], tcp=setup.scan_data_tcp_idx[i])
    interpolated_temperature_data = dsx_temperature_data.interp(time=data.time)
    list_dsx_scan_data += [xr.merge([interpolated_temperature_data, data])]

# Define coordinate systems


### Fixed systems
Important note: Measured data is not perfectly orthogonal. We use the cross product to get a z axis which is perfectly orthogonal (in floating point precision) and recalculate a perfectly orthogonal y-axis. There are certainly better ways to address this problem.

In [7]:
# Calculate orthogonal z axis and construct reference coordinate system from x and z axis
vec_x = setup.offset_ox_ref - setup.origin_ref
vec_y = setup.offset_oy_ref - setup.origin_ref
vec_z = np.cross(vec_x, vec_y)

cs_ref_in_base = tf.LocalCoordinateSystem.construct_from_xz_and_orientation(vec_x, vec_z, origin=setup.origin_ref)

cs_sp_in_ref = setup.cs_sp_in_ref
cs_torch_tcp_in_flange = setup.cs_torch_tcp_in_flange
cs_scanner_tcp_in_flange = setup.cs_scanner_tcp_in_flange
cs_cam_in_flange = setup.cs_cam_in_flange
cs_scan_in_scanner_tcp = setup.cs_scan_in_scanner_tcp
cs_scanner_schematic = tf.LocalCoordinateSystem(origin=[0, 0, -260])
list_cs_temp_in_ref = setup.list_cs_temp_in_ref

### Transformed systems

In [8]:
cs_sp_in_base = cs_sp_in_ref + cs_ref_in_base
cs_scanner_tcp_in_torch_tcp = cs_scanner_tcp_in_flange - cs_torch_tcp_in_flange
cs_cam_in_torch_tcp = cs_cam_in_flange - cs_torch_tcp_in_flange
cs_torch_tcp_in_scanner_tcp = cs_torch_tcp_in_flange - cs_scanner_tcp_in_flange
cs_cam_in_scanner_tcp = cs_cam_in_flange - cs_scanner_tcp_in_flange
list_cs_temp_in_sp = []
for i in range(len(list_cs_temp_in_ref)):
    list_cs_temp_in_sp += [list_cs_temp_in_ref[i] - cs_sp_in_ref]

### Functions for variable systems

In [9]:
def get_cs_scanner_tcp_in_sp(scan_data, index):
    angles = scan_data.scan_tcp.sel(tcp_variable=["Rx", "Ry", "Rz"]).data[index]
    orientation = rot.from_euler(angles=angles, seq="xyz", degrees=True).as_matrix()
    coordinates = scan_data.scan_tcp.sel(tcp_variable=["X", "Y", "Z"]).data[index]
    cs_scanner_tcp_in_base = tf.LocalCoordinateSystem(basis=orientation, origin=coordinates)
    return cs_scanner_tcp_in_base - cs_ref_in_base - cs_sp_in_ref


def get_cs_scan_in_sp(scan_data, index):
    return cs_scan_in_scanner_tcp + get_cs_scanner_tcp_in_sp(scan_data, index)

# Define data transformations

In [10]:
def coordinates_to_child_from_parent(coordinates, cs_child):
    rotation = cs_child.orientation.transpose()
    translation = cs_child.location[:, np.newaxis]
    return np.matmul(rotation, coordinates - translation)


def coordinates_to_parent_from_child(coordinates, cs_child):
    rotation = cs_child.orientation
    translation = cs_child.location[:, np.newaxis]
    return np.matmul(rotation, coordinates) + translation

# Transform data to specimen coordinate system

In [11]:
def transform_scan_data_to_sp(scan_data_in_scan):
    scanned_profiles_in_sp = []
    num_profiles_per_scan = scan_data_in_scan.profile.size
    for i in range(num_profiles_per_scan):
        scanned_profile_in_scan = scan_data_in_scan.scan_line.data[i].transpose()
        cs_scan_in_sp = get_cs_scan_in_sp(scan_data_in_scan, i)
        scanned_profiles_in_sp += [coordinates_to_parent_from_child(scanned_profile_in_scan, cs_scan_in_sp)]
    return np.array(scanned_profiles_in_sp, float)


list_scanned_profiles_in_sp = [transform_scan_data_to_sp(list_dsx_scan_data[0])]
list_scanned_profiles_in_sp += [transform_scan_data_to_sp(list_dsx_scan_data[1])]

# Create theoretical geometry

In [12]:
# create points
pt_0 = [150, 8]
pt_1_1 = [np.tan(setup.groove_angle_start / 360 * np.pi) * 8, 8]
pt_1_2 = [np.tan(setup.groove_angle_end / 360 * np.pi) * 8, 8]
pt_2 = [0, 0]
pt_3 = [150, 0]

# create shapes
shape_p1_r = geo.Shape().add_line_segments([pt_0, pt_1_1, pt_2])
shape_p2_r = geo.Shape().add_line_segments([pt_0, pt_1_2, pt_2])
shape_p1_l = shape_p1_r.reflect([1, 0])
shape_p2_l = shape_p2_r.reflect([1, 0])

# create profiles
profile_1 = geo.Profile([shape_p1_l, shape_p1_r])
profile_2 = geo.Profile([shape_p2_l, shape_p2_r])

# create variable profile
variable_profile = geo.VariableProfile([profile_1, profile_2], [0, 1], [geo.linear_profile_interpolation_sbs])

# create trace
trace = geo.Trace(geo.LinearHorizontalTraceSegment(350))

# create geometry
geometry = geo.Geometry(variable_profile, trace)

# rasterize profiles
profile_1_data = profile_1.rasterize(4)
profile_2_data = profile_2.rasterize(4)

# rasterize geometry
geometry_data_in_sp = geometry.rasterize(profile_raster_width=4, trace_raster_width=4)

# calculate triangles
triangles_geometry = Delaunay(geometry_data_in_sp.transpose()[:, :2]).simplices

# transform data to specimen coordinate system
vertices_geometry = np.matmul(tf.rotation_matrix_x(-np.pi / 2), geometry_data_in_sp)

# Create sequence

In [13]:
# first scan
num_timesteps_scan_0 = 100
num_scans_layer_0 = list_dsx_scan_data[0].profile.size
indices = np.arange(0, num_scans_layer_0 + 0.5, num_scans_layer_0 / num_timesteps_scan_0)
indices = np.array(np.round(indices), int)
list_idx_scan = [np.array(np.round(indices), int)]

# welding phase
num_timesteps_welding = 200
idx_start_scan_layer_1 = np.argmax(dsx_relevant_machine_data.trigScan2_prog.data > 0.1)
num_welding_data = idx_start_scan_layer_1
indices = np.arange(0, num_welding_data + 0.5, num_welding_data / num_timesteps_welding)
list_idx_welding = np.array(np.round(indices), int)
num_shown_video_frames = (dsx_relevant_machine_data.trigSchweissen.data[list_idx_welding] > 0).sum()

num_timesteps_scan_1 = 100
num_scans_layer_1 = list_dsx_scan_data[1].profile.size
indices = np.arange(0, num_scans_layer_1 + 0.5, num_scans_layer_1 / num_timesteps_scan_1)
indices = np.array(np.round(indices), int)
list_idx_scan += [np.array(np.round(indices), int)]


num_timesteps_total = num_timesteps_scan_0 + num_timesteps_welding + num_timesteps_scan_1

# Extract video frames

In [14]:
video_frames = np.array(np.load("./hd_05.frames.npy"))
num_video_frames = video_frames.shape[0]
delta_frame = num_video_frames / num_shown_video_frames
list_indices_frames = np.arange(0, num_video_frames - 0.5, delta_frame)
list_indices_frames = np.array(np.round(list_indices_frames), int)

video_frames = video_frames[list_indices_frames]

# Visualization

## Create meshes

### Cones

In [15]:
# create vertices and triangles
[vertices_cone, triangles_cone] = bm.create_cone_mesh(40)

# move tip to coordinate origin
vertices_cone = vertices_cone + np.array([0, 0, -1])[:, np.newaxis]

# scale
scale_cone = [15, 15, 30]
scale_mat_cone = [[scale_cone[0], 0, 0], [0, scale_cone[1], 0], [0, 0, scale_cone[2]]]
vertices_cone = np.matmul(scale_mat_cone, vertices_cone)
vertices_camera = vertices_cone + np.array([0, 0, 200])[:, np.newaxis]

### Cylinder

In [16]:
# create vertices and triangles
[vertices_cylinder, triangles_cylinder] = bm.create_unit_cylinder_mesh(40, 1)

# scale
scale_cylinder = [5, 5, 1]
scale_mat_cylinder = [[scale_cylinder[0], 0, 0], [0, scale_cylinder[1], 0], [0, 0, scale_cylinder[2]]]
vertices_cylinder = np.matmul(scale_mat_cylinder, vertices_cylinder)

### Weld

In [17]:
# Get reduced data of second scan
list_scp_weld = list_scanned_profiles_in_sp[1][::5, :, ::5]

# join all profiles
list_scp_weld = np.swapaxes(list_scp_weld, 1, 2)
num_total_points = list_scp_weld.shape[0] * list_scp_weld.shape[1]
list_scp_weld = np.reshape(list_scp_weld, (num_total_points, 3))

# Select data that is roughly part of the weld and not the scanned geometry
boolean_array = (list_scp_weld[:, 2] < 5) & (list_scp_weld[:, 2] > -5) & (list_scp_weld[:, 1] > -1)
vertices_weld = list_scp_weld[boolean_array]


# triangulate
triangles_weld = Delaunay(vertices_weld[:, [0, 2]]).simplices

vertices_weld = np.swapaxes(vertices_weld, 0, 1)

### Bundle data

In [18]:
class MeshData:
    def __init__(self, name, vertices, triangles, color, lcs=tf.LocalCoordinateSystem(), scale=None):
        self.name = name
        self.vertices = copy.deepcopy(vertices)
        self.triangles = triangles
        self.color = color
        self.lcs = lcs
        if scale is None:
            scale = [1, 1, 1]
        self.scale = scale
        self.visible = []
        self.sequence_data = []


meshes = [
    MeshData("geometry", vertices_geometry, triangles_geometry, [0.9, 0.9, 0.9]),
    MeshData("scanner", vertices_cone, triangles_cone, [1, 0, 0]),
    MeshData("torch", vertices_cone, triangles_cone, [0, 1, 0]),
    MeshData("thermoelement 0", vertices_cylinder, triangles_cylinder, [0, 0, 1], list_cs_temp_in_sp[0]),
    MeshData("thermoelement 1", vertices_cylinder, triangles_cylinder, [0, 0, 1], list_cs_temp_in_sp[1]),
    MeshData("thermoelement 2", vertices_cylinder, triangles_cylinder, [0, 0, 1], list_cs_temp_in_sp[2]),
    MeshData("camera", vertices_camera, triangles_cone, [1, 1, 0]),
    MeshData("weld", vertices_weld, triangles_weld, [1, 1, 0]),
]

### Create mesh sequence data

In [19]:
def temperature_to_color(temperature):
    temperature = np.clip(temperature, 0, 600)
    if temperature < 300:
        weight = temperature / 300
        return [0 + weight, 1, 0]
    else:
        weight = (temperature - 300) / 300
        return [1, 1 - weight, 0]


def get_transformed_vertices(vertex_data_mesh, cs_transformation,  scale):
    scale_mat = [[scale[0], 0, 0], [0, scale[1], 0], [0, 0, scale[2]]]
    vertex_data_trans = np.matmul(scale_mat, vertex_data_mesh)
    vertex_data_trans = coordinates_to_parent_from_child(vertex_data_trans, cs_transformation)
    return vertex_data_trans

def add_sequence_data(meshes):
    for i in range(len(meshes)):
        meshes[i].sequence_data += [get_transformed_vertices(meshes[i].vertices, meshes[i].lcs,  meshes[i].scale)]
    


def update_meshes(
    meshes, cs_scanner_in_sp, cs_torch_in_sp, cs_camera_in_sp, list_cs_temp_in_sp, torch_color, temperatures, weld_size
):
    for i in range(len(meshes)):
        if meshes[i].name == "scanner":
            meshes[i].lcs = cs_scanner_in_sp
        elif meshes[i].name == "torch":
            meshes[i].lcs = cs_torch_in_sp
            color_array = []
            for j in range(meshes[i].vertices.shape[1]):
                color_array += [torch_color]
            meshes[i].color += [color_array]
        elif meshes[i].name == "camera":
            meshes[i].lcs = cs_camera_in_sp
        elif meshes[i].name == "thermoelement 0":
            meshes[i].scale[2] = temperatures[0] / 2
            color_array = []
            for j in range(meshes[i].vertices.shape[1]):
                color_array += [temperature_to_color(temperatures[0])]
            meshes[i].color += [color_array]
        elif meshes[i].name == "thermoelement 1":
            meshes[i].scale[2] = temperatures[1] / 2
            color_array = []
            for j in range(meshes[i].vertices.shape[1]):
                color_array += [temperature_to_color(temperatures[1])]
            meshes[i].color += [color_array]
        elif meshes[i].name == "thermoelement 2":
            meshes[i].scale[2] = temperatures[2] / 2
            color_array = []
            for j in range(meshes[i].vertices.shape[1]):
                color_array += [temperature_to_color(temperatures[2])]
            meshes[i].color += [color_array]
        
        if meshes[i].name == "weld":
            if weld_size <= 0:
                meshes[i].visible += [False]
            else:
                meshes[i].visible += [True]
            meshes[i].vertices[0] = np.clip(vertices_weld[0], 0, weld_size)
        else:
            meshes[i].visible += [True]
            

def welding_phase_get_coordinate_systems(wd_idx, idx_switch_cs):
    x_angles_base = machine_angles_degree_base[:, wd_idx]
    x_coordinates_base = machine_coordinates_base[:, wd_idx]
    x_orientation_base = rot.from_euler(angles=x_angles_base, seq="xyz", degrees=True).as_matrix()
    cs_x_in_base = tf.LocalCoordinateSystem(basis=x_orientation_base, origin=x_coordinates_base)
    cs_x_in_sp = cs_x_in_base - cs_ref_in_base - cs_sp_in_ref

    if wd_idx > idx_switch_cs:
        cs_torch_in_sp = cs_torch_tcp_in_scanner_tcp + cs_x_in_sp
        cs_scanner_in_sp = cs_scanner_schematic + cs_x_in_sp
        cs_camera_in_sp = cs_cam_in_torch_tcp + cs_torch_in_sp
    else:
        cs_torch_in_sp = cs_x_in_sp
        cs_scanner_in_sp = cs_scanner_schematic + cs_scanner_tcp_in_torch_tcp + cs_torch_in_sp
        cs_camera_in_sp = cs_cam_in_torch_tcp + cs_torch_in_sp
    return [cs_torch_in_sp, cs_scanner_in_sp, cs_camera_in_sp]


def add_sequence_data_welding_phase(local_idx, idx_switch_cs, dsx_relevant_machine_data):
    wd_idx = list_idx_welding[local_idx]

    [cs_torch_in_sp, cs_scanner_in_sp, cs_camera_in_sp] = welding_phase_get_coordinate_systems(wd_idx, idx_switch_cs)

    torch_color = [0, 1, 0]
    if dsx_relevant_machine_data.trigSchweissen.data[wd_idx] > 0:
        brightness = local_idx % 2 
        torch_color = [brightness, brightness, 1]
    temperatures = [
        dsx_relevant_machine_data.MH24_T01.data[wd_idx],
        dsx_relevant_machine_data.MH24_T02.data[wd_idx],
        dsx_relevant_machine_data.MH24_T03.data[wd_idx],
    ]

    num_video_frames = video_frames.shape[0]

    if local_idx < video_frames.shape[0]:
        weld_size = cs_torch_in_sp.location[0] + 10
    else:
        weld_size = 350

    update_meshes(
        meshes,
        cs_scanner_in_sp,
        cs_torch_in_sp,
        cs_camera_in_sp,
        list_cs_temp_in_sp,
        torch_color,
        temperatures,
        weld_size,
    )
    add_sequence_data(meshes)
         


def add_sequence_data_scan_phase(scan_data, scanned_profiles_in_sp, idx, num_timesteps_scan, show_weld):
    cs_scanner_tcp_in_sp = get_cs_scanner_tcp_in_sp(scan_data, idx)
    cs_scanner_in_sp = cs_scanner_schematic + cs_scanner_tcp_in_sp
    cs_torch_in_sp = cs_torch_tcp_in_scanner_tcp + cs_scanner_tcp_in_sp
    cs_camera_in_sp = cs_cam_in_scanner_tcp + cs_scanner_tcp_in_sp

    temperatures = [scan_data.MH24_T01.data[idx], scan_data.MH24_T02.data[idx], scan_data.MH24_T03.data[idx]]

    torch_color = [0, 1, 0]

    if show_weld == True:
        weld_size = 350
    else:
        weld_size = 0

    update_meshes(
        meshes,
        cs_scanner_in_sp,
        cs_torch_in_sp,
        cs_camera_in_sp,
        list_cs_temp_in_sp,
        torch_color,
        temperatures,
        weld_size,
    )
    add_sequence_data(meshes)
    

def create_mesh_sequence_data(
    meshes, num_timesteps_scan_0, num_timesteps_welding, num_timesteps_scan_1, list_dsx_scan_data, list_scanned_profiles_in_sp
):
    for i in range(len(meshes)):
        meshes[i].sequence_data = []
        if meshes[i].name in ["torch", "thermoelement 0", "thermoelement 1", "thermoelement 2"]:
            meshes[i].color = []
    idx_switch_cs = np.argmax(dsx_machine_position_data.FB_Rz.dropna("time").diff("time").data)
    num_timesteps_total = num_timesteps_scan_0 + num_timesteps_welding + num_timesteps_scan_1
    for i in range(num_timesteps_total):
        if i < num_timesteps_scan_0:
            scn_idx = list_idx_scan[0][i]
            add_sequence_data_scan_phase(list_dsx_scan_data[0], list_scanned_profiles_in_sp[0], scn_idx, num_timesteps_scan_0, False)
        elif i < num_timesteps_scan_0 + num_timesteps_welding:
            local_idx = i - num_timesteps_scan_0
            add_sequence_data_welding_phase(local_idx, idx_switch_cs, dsx_relevant_machine_data)
        else:
            local_idx = i - num_timesteps_scan_0 - num_timesteps_welding
            scn_idx = list_idx_scan[1][local_idx]
            add_sequence_data_scan_phase(list_dsx_scan_data[1], list_scanned_profiles_in_sp[1], scn_idx, num_timesteps_scan_1, True)
    for i in range(len(meshes)):
        meshes[i].sequence_data = np.array(meshes[i].sequence_data)
        meshes[i].sequence_data = np.swapaxes(meshes[i].sequence_data,0,1)
        

create_mesh_sequence_data(meshes, num_timesteps_scan_0, num_timesteps_welding, num_timesteps_scan_1, list_dsx_scan_data, list_scanned_profiles_in_sp)

In [20]:
#meshes[1].sequence_data.shape
#meshes[5].color

## Functions

### Thermoelement color

### Mesh and scatter plot data 

In [21]:
def add_mesh(mesh):
    ipv.plot_trisurf(mesh.sequence_data[0], mesh.sequence_data[1], mesh.sequence_data[2], triangles=mesh.triangles, color=mesh.color)


def add_scatter(data, color):
    ipv.scatter(data[0], data[1], data[2], size=1, marker="sphere", color=color)


def transform_and_update_mesh(vertex_data_mesh, cs_transformation, ipvfig, ipv_idx, scale, visible):
    scale_mat = [[scale[0], 0, 0], [0, scale[1], 0], [0, 0, scale[2]]]
    vertex_data_trans = np.matmul(scale_mat, vertex_data_mesh)
    vertex_data_trans = coordinates_to_parent_from_child(vertex_data_trans, cs_transformation)
    ipvfig.meshes[ipv_idx].x = vertex_data_trans[0]
    ipvfig.meshes[ipv_idx].y = vertex_data_trans[1]
    ipvfig.meshes[ipv_idx].z = vertex_data_trans[2]
    ipvfig.meshes[ipv_idx].visible = visible


def update_meshes(
    meshes, cs_scanner_in_sp, cs_torch_in_sp, cs_camera_in_sp, list_cs_temp_in_sp, torch_color, temperatures, weld_size
):
    for i in range(len(meshes)):
        if meshes[i].name == "scanner":
            meshes[i].lcs = cs_scanner_in_sp
        elif meshes[i].name == "torch":
            meshes[i].lcs = cs_torch_in_sp
            meshes[i].color = torch_color
        elif meshes[i].name == "camera":
            meshes[i].lcs = cs_camera_in_sp
        elif meshes[i].name == "thermoelement 0":
            meshes[i].scale[2] = temperatures[0] / 2
            meshes[i].color = temperature_to_color(temperatures[0])
        elif meshes[i].name == "thermoelement 1":
            meshes[i].scale[2] = temperatures[1] / 2
            meshes[i].color = temperature_to_color(temperatures[1])
        elif meshes[i].name == "thermoelement 2":
            meshes[i].scale[2] = temperatures[2] / 2
            meshes[i].color = temperature_to_color(temperatures[2])
        elif meshes[i].name == "weld":
            if weld_size <= 0:
                meshes[i].visible = False
            else:
                meshes[i].visible = True
            meshes[i].vertices[0] = np.clip(vertices_weld[0], 0, weld_size)


def update_figures(
    ipvfig,
    meshes,
    cs_scanner_in_sp,
    cs_torch_in_sp,
    cs_cam_in_sp,
    list_cs_temp_in_sp,
    torch_color,
    temperatures,
    labels,
):
    for i in range(len(meshes)):
        transform_and_update_mesh(meshes[i].vertices, meshes[i].lcs, ipvfig, i, meshes[i].scale, meshes[i].visible)
        ipvfig.meshes[i].color = meshes[i].color
    for i in range(len(temperatures)):
        labels[i].value = "Thermoelement " + str(i) + ": " + str(np.round(temperatures[i], decimals=1)) + " °C"

# Create controls and widgets

In [22]:
def create_widgets():
    # plotting
    ipvfig = ipv.figure(width=800, height=600)
    ipv.xlim(0, 350)
    ipv.ylim(-20, 330)
    ipv.zlim(-175, 175)

    out = widgets.Output(layout={"border": "0px solid black"})
    with out:
        fig = plt.figure()
        fig.canvas.layout.width = "800px"
        fig.canvas.layout.height = "600px"
        gs = fig.add_gridspec(1, 1)
        ax_0 = fig.add_subplot(gs[0, 0])

    # time controls
    play = widgets.Play(
        value=0, min=0, max=num_timesteps_total, step=1, interval=1000/target_frame_rate, description="Press play", disabled=False
    )
    time_slider = IntSlider(min=0, max=num_timesteps_total, step=1, description="time", continuous_update=True)
    widgets.jslink((play, "value"), (time_slider, "value"))

    # thermoelement labels
    labels = [Label("Thermoelement 1: "), Label("Thermoelement 2: "), Label("Thermoelement 3: ")]

    # boxes
    upper_box = HBox()
    lower_box = HBox()
    info_box = VBox(labels, layout=Layout(top="200px", width="200px"))
    box = VBox([upper_box, lower_box])

    # add children to boxes
    upper_box.children = [info_box, ipvfig, out]
    lower_box.children = [play, time_slider]
    return [box, ipvfig, labels, ax_0, time_slider]

### Add meshes and scatters

In [23]:
def create_plots(meshes):
    scanned_profiles = np.concatenate((list_scanned_profiles_in_sp[0], list_scanned_profiles_in_sp[1]), axis=0)
    scanned_profiles = np.swapaxes(scanned_profiles, 0, 1)
    add_scatter(scanned_profiles, [1, 0, 0])

    for i in range(len(meshes)):
        add_mesh(meshes[i])

### Update meshes

In [24]:
num_timesteps_total

400

### Update scan sequence data

In [25]:
def scan_phase_plot_profile(scanner_position, scanned_profile):
    # create interpolated profile
    weight = scanner_position / 350
    profile_local = geo.linear_profile_interpolation_sbs(profile_1, profile_2, weight)
    profile_local_data = profile_local.rasterize(20)

    # plot everything - rasterized data must be plotted in two halves because of the order of points
    ax_0.clear()
    num_raster_points = profile_local_data.shape[1]
    nhalf = int(num_raster_points / 2)
    ax_0.plot(profile_local_data[0, 1:nhalf], profile_local_data[1, 1:nhalf], "b", label="ideal")
    ax_0.plot(profile_local_data[0, nhalf:], profile_local_data[1, nhalf:], "b")
    ax_0.plot(scanned_profile[2, :], scanned_profile[1, :], "r", label="scan")

    # format plot
    ax_0.axis("auto")  # Without this, the shown video will cause a rescaling of the plot
    ax_0.set_title("profile at x = " + str(np.round(scanner_position, decimals=1)) + "mm")
    ax_0.set_xlim([-20, 20])
    ax_0.set_ylim([0, 12])
    ax_0.set_xlabel("z in mm")
    ax_0.set_ylabel("y in mm")
    ax_0.legend()


def visualize_scan_phase(scan_data, scanned_profiles_in_sp, idx,time, num_timesteps_scan, show_weld, idx_offset):
    cs_scanner_tcp_in_sp = get_cs_scanner_tcp_in_sp(scan_data, idx)
    cs_scanner_in_sp = cs_scanner_schematic + cs_scanner_tcp_in_sp
    cs_torch_in_sp = cs_torch_tcp_in_scanner_tcp + cs_scanner_tcp_in_sp
    cs_camera_in_sp = cs_cam_in_scanner_tcp + cs_scanner_tcp_in_sp

    scan_phase_plot_profile(cs_scanner_in_sp.location[0], scanned_profiles_in_sp[idx])

    ipvfig.scatters[0].sequence_index = idx.item() + idx_offset
    ipvfig.scatters[0].visible = True
    temperatures = [scan_data.MH24_T01.data[idx], scan_data.MH24_T02.data[idx], scan_data.MH24_T03.data[idx]]
    for i in range(len(ipvfig.meshes)):
        ipvfig.meshes[i].sequence_index = time
        ipvfig.meshes[i].visible = meshes[i].visible[time]

    torch_color = [0, 1, 0]
    if show_weld == True:
        weld_size = 350
    else:
        weld_size = 0
 

### Update welding sequence data

In [26]:
def visualize_welding_phase(local_idx,time, idx_switch_cs, dsx_relevant_machine_data, vertices_weld):
    wd_idx = list_idx_welding[local_idx]

    torch_color = [0, 1, 0]
    if dsx_relevant_machine_data.trigSchweissen.data[wd_idx] > 0:
        brightness = (local_idx % 5) / 4
        torch_color = [brightness, brightness, 1]
    temperatures = [
        dsx_relevant_machine_data.MH24_T01.data[wd_idx],
        dsx_relevant_machine_data.MH24_T02.data[wd_idx],
        dsx_relevant_machine_data.MH24_T03.data[wd_idx],
    ]

    num_video_frames = video_frames.shape[0]

    for i in range(len(ipvfig.meshes)):
        ipvfig.meshes[i].sequence_index = time
        ipvfig.meshes[i].visible = meshes[i].visible[time]
    if local_idx < video_frames.shape[0]:
        ax_0.clear()
        ax_0.imshow(video_frames[local_idx], cmap="gray", vmin=0, vmax=255)

In [27]:
[box, ipvfig, labels, ax_0, time_slider] = create_widgets()
create_plots(meshes)


idx_switch_cs = np.argmax(dsx_machine_position_data.FB_Rz.dropna("time").diff("time").data)


def update_output(time):
    if time < num_timesteps_scan_0:
        scn_idx = list_idx_scan[0][time]        
        visualize_scan_phase(
            list_dsx_scan_data[0], list_scanned_profiles_in_sp[0], scn_idx,time, num_timesteps_scan_0, False, 0
        )
    elif time < num_timesteps_scan_0 + num_timesteps_welding:
        ipvfig.scatters[0].visible = False
        local_idx = time - num_timesteps_scan_0
        visualize_welding_phase(local_idx,time, idx_switch_cs, dsx_relevant_machine_data, vertices_weld)
    else:
        scn_idx = list_idx_scan[1][time - num_timesteps_scan_0 - num_timesteps_welding]
        visualize_scan_phase(
            list_dsx_scan_data[1],
            list_scanned_profiles_in_sp[1],
            scn_idx,time,
            num_timesteps_scan_1,
            True,
            num_scans_layer_0,
        )

ipv.xlim(0, 350)
ipv.ylim(-20, 330)
ipv.zlim(-175, 175)
ipvfig.animation = 1000/target_frame_rate
update_output(110)
output = interactive_output(update_output, dict(time=time_slider))
display(box)
_ = ipv.view(elevation=30, azimuth=140, distance=1.7)

In [28]:
ipvfig.meshes[7].visible

False

In [29]:
vertices_weld.shape

(3, 7201)

In [30]:
ipvfig.meshes[0].

SyntaxError: invalid syntax (<ipython-input-30-1438bf8c9dc4>, line 1)